In [ ]:
import numpy as np
import pandas as pd
import os
import datetime as dt
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import files
 
 
uploaded = files.upload()

Saving Year 2009-2010.csv to Year 2009-2010.csv


In [ ]:
import io
 
raw_df = pd.read_csv(io.BytesIO(uploaded['Year 2009-2010.csv']), encoding= 'unicode_escape')
raw_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25,13085.0,United Kingdom


In [ ]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      525461 non-null  object 
 1   StockCode    525461 non-null  object 
 2   Description  522533 non-null  object 
 3   Quantity     525461 non-null  int64  
 4   InvoiceDate  525461 non-null  object 
 5   Price        525461 non-null  float64
 6   Customer ID  417534 non-null  float64
 7   Country      525461 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.1+ MB


In [ ]:
raw_df.shape

(525461, 8)

In [ ]:
raw_df.isnull().sum()


Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [ ]:
df = raw_df.dropna()

In [ ]:
df_Invoice = pd.DataFrame({"Invoice":[row for row in df["Invoice"].values if "C"  not in str(row)]})
df_Invoice.head()

,Invoice
0,489434
1,489434
2,489434
3,489434
4,489434


In [ ]:
df_Invoice = df_Invoice.drop_duplicates("Invoice")

df = df.merge(df_Invoice, on = "Invoice")

In [ ]:
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

In [ ]:
df.tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
407690,538171,22271,FELTCRAFT DOLL ROSIE,2,12/9/2010 20:01,2.95,17530.0,United Kingdom
407691,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,12/9/2010 20:01,3.75,17530.0,United Kingdom
407692,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,12/9/2010 20:01,3.75,17530.0,United Kingdom
407693,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,12/9/2010 20:01,3.75,17530.0,United Kingdom
407694,538171,21931,JUMBO STORAGE BAG SUKI,2,12/9/2010 20:01,1.95,17530.0,United Kingdom


In [ ]:
df.nunique()

Invoice        19213
StockCode       4017
Description     4444
Quantity         343
InvoiceDate    18008
Price            448
Customer ID     4312
Country           37
dtype: int64

In [ ]:
df_product = df[["Description","StockCode"]].drop_duplicates()
df_product = df_product.groupby(["Description"]).agg({"StockCode":"count"}).reset_index()

df_product.rename(columns={'StockCode':'StockCode_Count'},inplace=True)

df_product = df_product.sort_values("StockCode_Count", ascending=False)
df_product = df_product[df_product["StockCode_Count"]>1]

df_product

,Description,StockCode_Count
4228,WHITE BAMBOO RIBS LAMPSHADE,3
1009,COLOURING PENCILS BROWN TUBE,3
2331,"METAL SIGN,CUPCAKE SINGLE HOOK",3
3270,RETRO PLASTIC POLKA TRAY,2
4213,WATERING CAN PINK BUNNY,2
1011,COLUMBIAN CANDLE RECTANGLE,2
1290,EAU DE NILE JEWELLED PHOTOFRAME,2
3268,RETRO PLASTIC DAISY TRAY,2
4212,WATERING CAN GREEN DINOSAUR,2
538,BLUE FLOCK GLASS CANDLEHOLDER,2


In [ ]:
df = df[~df["Description"].isin(df_product["Description"])]

print(df.StockCode.nunique())
print(df.Description.nunique())

3969
4419


In [ ]:
df_product = df[["Description","StockCode"]].drop_duplicates()
df_product = df_product.groupby(["StockCode"]).agg({"Description":"count"}).reset_index()
df_product.rename(columns={'Description':'Description_Count'},inplace=True)
df_product = df_product.sort_values("Description_Count", ascending=False)
df_product = df_product[df_product["Description_Count"] > 1] 


df_product.head()

,StockCode,Description_Count
1528,22344,4
213,20685,4
1563,22384,4
1200,21955,4
1530,22346,4


In [ ]:
df = df[~df["StockCode"].isin(df_product["StockCode"])]

print(df.StockCode.nunique())
print(df.Description.nunique())

3550
3550


In [ ]:
df = df[~df["StockCode"].str.contains("POST", na=False)]

In [ ]:
df['TotalPrice'] = df['Quantity'] * df['Price']

In [ ]:
df.shape[1]

9

In [ ]:
cltv_c = df.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),  
                                        'TotalPrice': lambda x: x.sum()})  
cltv_c.head()

,Invoice,TotalPrice
Customer ID,,
12346.0,2,76.23
12347.0,2,989.36
12348.0,1,170.40
12349.0,2,1239.44
12351.0,1,238.08


In [ ]:
cltv_c.columns = ['Total_Transaction',  'Total_Price']

cltv_c.reset_index(inplace=True)

cltv_c.head()

,Customer ID,Total_Transaction,Total_Price
0,12346.0,2,76.23
1,12347.0,2,989.36
2,12348.0,1,170.40
3,12349.0,2,1239.44
4,12351.0,1,238.08


In [ ]:
cltv_c['Customer ID'] = cltv_c['Customer ID'].astype(int)

In [ ]:
cltv_c.head()


,Customer ID,Total_Transaction,Total_Price
0,12346,2,76.23
1,12347,2,989.36
2,12348,1,170.40
3,12349,2,1239.44
4,12351,1,238.08


In [ ]:
cltv_c['Average_Order_Value'] = cltv_c['Total_Price'] / cltv_c['Total_Transaction']
cltv_c.head()

,Customer ID,Total_Transaction,Total_Price,Average_Order_Value
0,12346,2,76.23,38.115
1,12347,2,989.36,494.680
2,12348,1,170.40,170.400
3,12349,2,1239.44,619.720
4,12351,1,238.08,238.080


In [ ]:
numofCustomers = cltv_c.shape[0]

cltv_c["Purchase_Frequency"] = cltv_c['Total_Transaction'] / numofCustomers
cltv_c.head()

,Customer ID,Total_Transaction,Total_Price,Average_Order_Value,Purchase_Frequency
0,12346,2,76.23,38.115,0.000468
1,12347,2,989.36,494.680,0.000468
2,12348,1,170.40,170.400,0.000234
3,12349,2,1239.44,619.720,0.000468
4,12351,1,238.08,238.080,0.000234


In [ ]:
repeat_rate = cltv_c[cltv_c.Total_Transaction > 1].shape[0] / numofCustomers 
churn_rate = 1 - repeat_rate

print(churn_rate)

0.33044901777362023


In [ ]:
cltv_c['Profit_Margin'] = cltv_c['Total_Price'] * 0.10
cltv_c.head()

,Customer ID,Total_Transaction,Total_Price,Average_Order_Value,Purchase_Frequency,Profit_Margin
0,12346,2,76.23,38.115,0.000468,7.623
1,12347,2,989.36,494.680,0.000468,98.936
2,12348,1,170.40,170.400,0.000234,17.040
3,12349,2,1239.44,619.720,0.000468,123.944
4,12351,1,238.08,238.080,0.000234,23.808


In [ ]:
cltv_c.head()

,Customer ID,Total_Transaction,Total_Price,Average_Order_Value,Purchase_Frequency,Profit_Margin
0,12346,2,76.23,38.115,0.000468,7.623
1,12347,2,989.36,494.680,0.000468,98.936
2,12348,1,170.40,170.400,0.000234,17.040
3,12349,2,1239.44,619.720,0.000468,123.944
4,12351,1,238.08,238.080,0.000234,23.808


In [ ]:
cltv_c['CLTV'] = cltv_c['Average_Order_Value'] * cltv_c["Purchase_Frequency"]
scaler = MinMaxScaler(feature_range=(0, 100))
cltv_c["Scaled_CLTV"] = scaler.fit_transform(cltv_c[["CLTV"]]) 
cltv_c.tail()

,Customer ID,Total_Transaction,Total_Price,Average_Order_Value,Purchase_Frequency,Profit_Margin,CLTV,Scaled_CLTV
4271,18283,6,447.12,74.5200,0.001403,44.712,0.104565,0.178279
4272,18284,1,367.48,367.4800,0.000234,36.748,0.085940,0.146414
4273,18285,1,419.20,419.2000,0.000234,41.920,0.098036,0.167107
4274,18286,2,990.79,495.3950,0.000468,99.079,0.231710,0.395808
4275,18287,4,2029.81,507.4525,0.000935,202.981,0.474698,0.811534


In [ ]:
cltv_c['Scaled_CLTV'].max()

100.0

In [ ]:
cltv_c.sort_values(by="Scaled_CLTV", ascending=False).head()

,Customer ID,Total_Transaction,Total_Price,Average_Order_Value,Purchase_Frequency,Profit_Margin,CLTV,Scaled_CLTV
4148,18102,79,249930.63,3163.678861,0.018475,24993.063,58.449633,100.000000
1615,14646,69,171126.59,2480.095507,0.016137,17112.659,40.020250,68.469439
1250,14156,94,129975.17,1382.714574,0.021983,12997.517,30.396438,52.004201
1818,14911,199,122791.33,617.041859,0.046539,12279.133,28.716401,49.129849
923,13694,90,109601.25,1217.791667,0.021048,10960.125,25.631724,43.852320


In [ ]:
cltv_c["Segment"] = pd.qcut(cltv_c["Scaled_CLTV"], 4, labels=["D", "C", "B", "A"])

cltv_c.groupby("Segment").agg( {"Total_Transaction": ["count", "mean", "sum"],
                                "Total_Price": ["mean", "sum"],
                                "Scaled_CLTV": "mean"})

Total_Transaction                    Total_Price               \
                    count       mean    sum         mean          sum   
Segment                                                                 
D                    1069   1.224509   1309   141.930328   151723.521   
C                    1069   1.966324   2102   386.677522   413358.271   
B                    1069   3.738073   3996   901.793088   964016.811   
A                    1069  10.557530  11286  4906.071797  5244590.751   

        Scaled_CLTV  
               mean  
Segment              
D          0.056168  
C          0.154095  
B          0.360199  
A          1.962365

In [ ]:
df = df.loc[df["Country"] == "United Kingdom",:]

In [ ]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [ ]:
num_cols  = [col for col in df.columns if df[col].dtype != "O" and "ID" not in col ]
num_cols

['Quantity', 'Price', 'TotalPrice']

In [ ]:
for col in num_cols:
    print(outlier_thresholds(df, col))

(-177.5, 298.5)
(-18.5, 31.5)
(-261.175, 437.025)


In [ ]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [ ]:
for col in num_cols:
    replace_with_thresholds(df, col)



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293424 entries, 0 to 407694
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      293424 non-null  object 
 1   StockCode    293424 non-null  object 
 2   Description  293424 non-null  object 
 3   Quantity     293424 non-null  float64
 4   InvoiceDate  293424 non-null  object 
 5   Price        293424 non-null  float64
 6   Customer ID  293424 non-null  float64
 7   Country      293424 non-null  object 
 8   TotalPrice   293424 non-null  float64
dtypes: float64(4), object(5)
memory usage: 22.4+ MB


In [ ]:
df['InvoiceDate'] = df['InvoiceDate'].astype('datetime64[ns]')

In [ ]:
performans_date =   df["InvoiceDate"].max() + dt.timedelta(days=2)

In [ ]:
cltv_df = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,           # recency
                                                         lambda date: (performans_date - date.min()).days],     # T 
                                         'Invoice': lambda num: num.nunique(),                                  # frequency
                                         'TotalPrice': lambda TotalPrice: TotalPrice.sum()})                    # total price

cltv_df.head()

InvoiceDate             Invoice TotalPrice
             <lambda_0> <lambda_1> <lambda>   <lambda>
Customer ID                                           
12346.0             118        284        2      76.23
12608.0               0         41        1     372.09
12745.0              87        211        2     486.85
12746.0               0        177        1     176.35
12747.0             363        369       16    4797.01

In [ ]:
cltv_df.columns = cltv_df.columns.droplevel(0)
cltv_df.columns = ['Recency', 'Tenure', 'Frequency', 'Monetary']

In [ ]:
cltv_df.head()

,Recency,Tenure,Frequency,Monetary
Customer ID,,,,
12346.0,118,284,2,76.23
12608.0,0,41,1,372.09
12745.0,87,211,2,486.85
12746.0,0,177,1,176.35
12747.0,363,369,16,4797.01


In [ ]:
cltv_df["Recency"] = cltv_df["Recency"] / 7
cltv_df["Tenure"] = cltv_df["Tenure"] / 7


In [ ]:
cltv_df["Monetary"] = cltv_df["Monetary"] / cltv_df["Frequency"]
 
# Monetary > 0 :
cltv_df = cltv_df[cltv_df["Monetary"] > 0]
cltv_df.head()

,Recency,Tenure,Frequency,Monetary
Customer ID,,,,
12346.0,16.857143,40.571429,2,38.115000
12608.0,0.000000,5.857143,1,372.090000
12745.0,12.428571,30.142857,2,243.425000
12746.0,0.000000,25.285714,1,176.350000
12747.0,51.857143,52.714286,16,299.813125


In [ ]:
cltv_df = cltv_df[(cltv_df['Frequency'] > 1)]

In [ ]:
!pip install lifetimes
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions


     |████████████████████████████████| 584 kB 9.9 MB/s 


In [ ]:
bgf = BetaGeoFitter(penalizer_coef=6)
bgf.fit(cltv_df['Frequency'],
        cltv_df['Recency'],
        cltv_df['Tenure']);

In [ ]:
bgf.conditional_expected_number_of_purchases_up_to_time(6 * 4,  #weekly
                                                        cltv_df['Frequency'],
                                                        cltv_df['Recency'],
                                                        cltv_df['Tenure']).sort_values(ascending=False).head(10)


Customer ID
17850.0    68.972548
12748.0    61.275535
15311.0    52.437848
13089.0    46.840595
14606.0    42.543744
13694.0    39.989707
17841.0    39.786718
18102.0    34.844328
15061.0    34.404066
15235.0    32.723615
dtype: float64

In [ ]:
cltv_df["Expected_Purchase_6_months"] = bgf.predict(6 * 4,
                                           cltv_df['Frequency'],
                                           cltv_df['Recency'],
                                           cltv_df['Tenure'])
cltv_df.head()

,Recency,Tenure,Frequency,Monetary,Expected_Purchase_6_months
Customer ID,,,,,
12346.0,16.857143,40.571429,2,38.115000,1.273086
12745.0,12.428571,30.142857,2,243.425000,1.699567
12747.0,51.857143,52.714286,16,299.813125,7.248171
12748.0,52.714286,53.142857,138,132.855109,61.275535
12749.0,17.285714,22.571429,4,557.257500,4.294805


In [ ]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['Frequency'], cltv_df['Monetary'])

<lifetimes.GammaGammaFitter: fitted with 2642 subjects, p: 3.89, q: 0.37, v: 3.82>

In [ ]:
ggf.conditional_expected_average_profit(cltv_df['Frequency'],
                                        cltv_df['Monetary']).sort_values(ascending=False).head(10)

Customer ID
14088.0    5675.810635
16749.0    2297.356017
18251.0    2101.240319
16684.0    2062.293931
17511.0    1951.214113
14091.0    1931.770710
17450.0    1884.568487
18102.0    1752.187215
15823.0    1563.770145
15786.0    1522.482277
dtype: float64

In [ ]:
cltv_df["Expected_Average_Profit_6_months"] = ggf.conditional_expected_average_profit(cltv_df['Frequency'],
                                                                             cltv_df['Monetary'])

In [ ]:
cltv_df.head()

,Recency,Tenure,Frequency,Monetary,Expected_Purchase_6_months,Expected_Average_Profit_6_months
Customer ID,,,,,,
12346.0,16.857143,40.571429,2,38.115000,1.273086,43.544308
12745.0,12.428571,30.142857,2,243.425000,1.699567,266.912219
12747.0,51.857143,52.714286,16,299.813125,7.248171,303.114872
12748.0,52.714286,53.142857,138,132.855109,61.275535,133.038650
12749.0,17.285714,22.571429,4,557.257500,4.294805,581.726501


In [ ]:
cltv_6_months = ggf.customer_lifetime_value(bgf,
                                   cltv_df['Frequency'],
                                   cltv_df['Recency'],
                                   cltv_df['Tenure'],
                                   cltv_df['Monetary'],
                                   time=6,  # 6 months
                                   freq="W",  
                                   discount_rate=0.01)

In [ ]:
cltv_6_months.head(2)

Customer ID
12346.0     58.162778
12745.0    475.948943
Name: clv, dtype: float64

In [ ]:
cltv_6_months = cltv_df.merge(cltv_6_months, on="Customer ID", how="left")
cltv_6_months.head()

,Recency,Tenure,Frequency,Monetary,Expected_Purchase_6_months,Expected_Average_Profit_6_months,clv
Customer ID,,,,,,,
12346.0,16.857143,40.571429,2,38.115000,1.273086,43.544308,58.162778
12745.0,12.428571,30.142857,2,243.425000,1.699567,266.912219,475.948943
12747.0,51.857143,52.714286,16,299.813125,7.248171,303.114872,2305.133479
12748.0,52.714286,53.142857,138,132.855109,61.275535,133.038650,8553.142872
12749.0,17.285714,22.571429,4,557.257500,4.294805,581.726501,2621.293368


In [ ]:
cltv_6_months.rename(columns={"clv": "CLTV_6_months"}, inplace=True)

In [ ]:
cltv_6_months.head()

,Recency,Tenure,Frequency,Monetary,Expected_Purchase_6_months,Expected_Average_Profit_6_months,CLTV_6_months
Customer ID,,,,,,,
12346.0,16.857143,40.571429,2,38.115000,1.273086,43.544308,58.162778
12745.0,12.428571,30.142857,2,243.425000,1.699567,266.912219,475.948943
12747.0,51.857143,52.714286,16,299.813125,7.248171,303.114872,2305.133479
12748.0,52.714286,53.142857,138,132.855109,61.275535,133.038650,8553.142872
12749.0,17.285714,22.571429,4,557.257500,4.294805,581.726501,2621.293368


In [ ]:
scaler = MinMaxScaler(feature_range=(0, 100))
scaler.fit(cltv_6_months[["CLTV_6_months"]])
cltv_6_months["Scaled_CLTV"] = scaler.transform(cltv_6_months[["CLTV_6_months"]])

In [ ]:
cltv_6_months.head()


,Recency,Tenure,Frequency,Monetary,Expected_Purchase_6_months,Expected_Average_Profit_6_months,CLTV_6_months,Scaled_CLTV
Customer ID,,,,,,,,
12346.0,16.857143,40.571429,2,38.115000,1.273086,43.544308,58.162778,0.090797
12745.0,12.428571,30.142857,2,243.425000,1.699567,266.912219,475.948943,0.742997
12747.0,51.857143,52.714286,16,299.813125,7.248171,303.114872,2305.133479,3.598510
12748.0,52.714286,53.142857,138,132.855109,61.275535,133.038650,8553.142872,13.352185
12749.0,17.285714,22.571429,4,557.257500,4.294805,581.726501,2621.293368,4.092062


In [ ]:
cltv_6_months["CLTV_Segment"] = pd.qcut(cltv_6_months["Scaled_CLTV"], 4, labels=["D", "C", "B", "A"])
cltv_6_months.reset_index(inplace=True)
cltv_6_months.sort_values(by="Scaled_CLTV", ascending=False).head(10)

,Customer ID,Recency,Tenure,Frequency,Monetary,Expected_Purchase_6_months,Expected_Average_Profit_6_months,CLTV_6_months,Scaled_CLTV,CLTV_Segment
2577,18102.0,53.285714,53.571429,79,1748.552722,34.844328,1752.187215,64058.002318,100.000000,A
418,13694.0,51.571429,53.142857,90,1188.202500,39.989707,1190.383481,49945.443441,77.969093,A
608,14088.0,2.000000,8.285714,2,5215.047500,5.665010,5675.810635,33734.356372,52.662205,A
2256,17450.0,10.000000,10.714286,7,1840.492857,14.802979,1884.568487,29269.126499,45.691600,A
2279,17511.0,52.714286,53.428571,28,1939.810357,12.444944,1951.214113,25477.599482,39.772704,A
928,14733.0,4.857143,5.571429,4,1439.787500,15.434094,1501.432778,24312.545631,37.953955,A
1861,16684.0,50.285714,52.714286,23,2047.630217,10.375282,2062.293931,22449.724077,35.045932,A
130,13089.0,52.571429,53.428571,106,449.525425,46.840595,450.248232,22127.646432,34.543142,A
1076,15061.0,53.000000,53.571429,78,577.609551,34.404066,578.858427,20895.026213,32.618916,A
1897,16754.0,38.428571,39.714286,29,1134.606897,17.226492,1141.100685,20624.326054,32.196330,A


In [ ]:
cltv_6_months["Expected_Average_Profit_6_months"].sum()

792588.8142570917

In [ ]:
cltv_6_months["Expected_Average_Profit_6_months"].mean()

299.9957661836071

In [ ]:
cltv_6_months["Expected_Purchase_6_months"].mean()

4.179702717689522

In [ ]:
cltv_6_months.groupby("CLTV_Segment").agg( {"Customer ID": "count",
                                            "Expected_Purchase_6_months": ["mean", "sum"],
                                            "Expected_Average_Profit_6_months": [ "mean", "sum"],
                                            "Scaled_CLTV": "mean"})

Customer ID Expected_Purchase_6_months               \
                   count                       mean          sum   
CLTV_Segment                                                       
D                    661                   1.654520  1093.637944   
C                    660                   2.701348  1782.889941   
B                    660                   3.983825  2629.324790   
A                    661                   8.376584  5536.921905   

             Expected_Average_Profit_6_months                Scaled_CLTV  
                                         mean            sum        mean  
CLTV_Segment                                                              
D                                  167.105710  110456.874069    0.394797  
C                                  248.669547  164121.901054    0.940951  
B                                  316.785718  209078.573622    1.712563  
A                                  467.369842  308931.465511    5.669899